In [1]:
from warnings import filters
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2  
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from glob import glob
import numpy as np

In [2]:
# Define image size and paths 
IMAGE_SIZE = [224, 224]
train_path = "C:\Users\narendra\Downloads\archive_extracted\Train"
valid_path = "C:\Users\narendra\Downloads\archive_extracted\Test"


In [3]:
# MobileNetV2 base model 
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=IMAGE_SIZE + [3])  


In [4]:
# Freeze all but the last 10 layers 
for layer in base_model.layers[:-10]:
    layer.trainable = False


In [5]:
# Additional layers 
x = base_model.output
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)  # Extract more spatial features
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Prevent overfitting
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)  # Another convolutional layer
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Flatten()(x)


In [6]:
# Dense layers with ReLU or LeakyReLU activation for classification 
x = Dense(128, activation='relu')(x)  # Consider LeakyReLU for vanishing gradient issues
x = Dropout(0.4)(x)  # Increased dropout rate
# Get the number of output classes based on training data folders
folders = glob('E://Train/*')


In [7]:
# Output layer with softmax activation for multi-class classification
prediction = Dense(len(folders), activation='softmax')(x)


In [8]:
# Create the model with the additional layers
model = Model(inputs=base_model.input, outputs=prediction)



In [9]:
# Compile the model with appropriate metrics and optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [10]:
# Data augmentation 
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   rotation_range=20,  # Add rotation
                                   width_shift_range=0.2,  # Add width/height shift
                                   height_shift_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)


In [11]:
# Flow data from directories with target size and batch size
training_set = train_datagen.flow_from_directory(train_path, target_size=IMAGE_SIZE, batch_size=32, class_mode='categorical')
test_set = test_datagen.flow_from_directory(valid_path, target_size=IMAGE_SIZE, batch_size=32, class_mode='categorical')


Found 89046 images belonging to 14 classes.
Found 22262 images belonging to 14 classes.


In [14]:
# Train the model with callbacks
r = model.fit(training_set,
              validation_data=test_set,
              epochs=10,
              steps_per_epoch=len(training_set),
              validation_steps=len(test_set),
              )


Epoch 1/10


C:\Users\Badari\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2783/2783 ━━━━━━━━━━━━━━━━━━━━ 2913s 1s/step - accuracy: 0.7066 - loss: 1.0543 - val_accuracy: 0.8601 - val_loss: 1.0772
Epoch 2/10
2783/2783 ━━━━━━━━━━━━━━━━━━━━ 0s 83us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00


C:\Users\Badari\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


Epoch 3/10
2783/2783 ━━━━━━━━━━━━━━━━━━━━ 2689s 965ms/step - accuracy: 0.9240 - loss: 0.2691 - val_accuracy: 0.9261 - val_loss: 0.3060
Epoch 4/10
2783/2783 ━━━━━━━━━━━━━━━━━━━━ 0s 27us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
2783/2783 ━━━━━━━━━━━━━━━━━━━━ 2593s 931ms/step - accuracy: 0.9604 - loss: 0.1394 - val_accuracy: 0.9675 - val_loss: 0.1377
Epoch 6/10
2783/2783 ━━━━━━━━━━━━━━━━━━━━ 0s 26us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
2783/2783 ━━━━━━━━━━━━━━━━━━━━ 2527s 907ms/step - accuracy: 0.9742 - loss: 0.0943 - val_accuracy: 0.9760 - val_loss: 0.0884
Epoch 8/10
2783/2783 ━━━━━━━━━━━━━━━━━━━━ 0s 23us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/10
2783/2783 ━━━━━━━━━━━━━━━━━━━━ 2527s 907ms/step - accuracy: 0.9801 - loss: 0.0737 - val_accuracy: 0.9780 - val_loss: 0.0800
Epoch 10/10
2783/27

In [17]:
# Save the model


model.save("crime_detection_model_optimized_1.h5")
print("Model saved successfully.")


Model saved successfully.


In [19]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Load the model
model = load_model("crime_detection_model_optimized_1.h5")

# Load the test data 
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    valid_path,
    target_size=(IMAGE_SIZE),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Do not shuffle for evaluation
)

# Evaluate model performance on the test data
test_loss, test_accuracy = model.evaluate(test_set)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

# Predict class labels for the test set
predictions = model.predict(test_set)
predicted_classes = np.argmax(predictions, axis=1)

# Get the true labels
true_labels = test_set.classes

# Get class labels
class_labels = list(test_set.class_indices.keys())

# Print classification report
print("Classification Report:")
print(classification_report(true_labels, predicted_classes, target_names=class_labels))

# Compute and print confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)


Found 22262 images belonging to 14 classes.


C:\Users\Badari\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


696/696 ━━━━━━━━━━━━━━━━━━━━ 406s 580ms/step - accuracy: 0.9795 - loss: 0.0813
Test loss: 0.08000743389129639
Test accuracy: 0.9780343174934387
696/696 ━━━━━━━━━━━━━━━━━━━━ 405s 580ms/step
Classification Report:
               precision    recall  f1-score   support

        Abuse       1.00      0.60      0.75        58
       Arrest       0.97      0.99      0.98       694
        Arson       0.97      0.94      0.96       542
      Assault       1.00      0.98      0.99       555
     Burglary       0.99      0.99      0.99      1535
    Explosion       0.93      0.98      0.95      1280
     Fighting       1.00      0.93      0.96       240
 NormalVideos       0.98      1.00      0.99     13110
RoadAccidents       1.00      0.96      0.98       541
      Robbery       0.95      1.00      0.97       154
     Shooting       1.00      0.93      0.97      1477
  Shoplifting       0.99      0.87      0.92      1496
     Stealing       1.00      0.96      0.98       389
    Vandalism    